<center>
  <h1>Playing with Python Bytecode</h1><br><br>
  <h3><a href="https://github.com/ssanderson/pycon-2016">https://github.com/ssanderson/pycon-2016</a>
  </h3><br>
</center>

In [5]:
# (Open to Scott standing at podium, on title slide. Joe is hiding in the audience.)
#
# Scott: Hey everyone, thank you all for coming out today. We've just received word 
#        from Joe and Scott via text message that they're not going to be able to 
#        make it today due to a freak scheduling accident.  But we know that you were
#        all super excited to learn about Python bytecode, so I'm gonna try to do my
#        best to give the talk in their place.
#
# Scott: So, I took a look at their abstract a minute ago...
# 
# Scott: "Playing with Python Bytecode"... in this talk we explain CPython's
#        internal code representation, demonstrating techniques for modifying
#        code objects.  We break down the attributes of Python's code type,
#        showing how one might construct a function or module "from scratch".
#
# Scott: Okay, so it sounds like this talk is supposed to be about mucking 
#        with compiled Python functions.  I guess that means we need a function 
#        to muck with.

def add(a, b):
    return a + b
add(1, 2)

3

In [6]:
# Scott: We've got our add() function. There must be some way for us to access
#        its bytecode if we're going to play with it.
#        All the magic stuff in Python starts and ends with double
#        underscores.  Maybe we can find the bytecode.

# add.__<TAB>
# add.__code__
# add.__code__.co_<TAB>
# add.__code__.co_argcount
# add.__code__.co_consts
add.__code__.co_code

# Scott: These don't really look like printable characters.
# Scott: Maybe we can understand these better if we look at the byte values as integers.

b'|\x00\x00|\x01\x00\x17S'

In [7]:
print(list(add.__code__.co_code))

# Scott: Hrm. There's definitely some structure here.  
#        I see (124, 0, 0) followed by (124, 1, 0). Maybe one of
#        these corresponds to 'a' and the other corresponds to 'b'?
#        (confused noises)
#
# Scott: I think I'm a little out of my depth here.  Does anyone in the audience    
#        know what this string means?
#
# Joe stands in audience, wearing a 'BYTECODE EXPERT' shirt.  
# He turns to face the audience, looking around to see if anyone 
# else will answer the call to action.
#    
# After a moment, he shrugs and says:
#
# Joe: Well, I happen to be a Certified Bytecode Expert.
#
# Scott: Great, let's get him up here. Can we get a mic for him? (Joe is already mic'ed)
#
# Joe: It's okay, I brought my own.

# (Joe scrambles up onto the stage.)

# Scott: Wait, how do you already have a micro--

# Joe: Let's keep on track here. You had the right idea about those LOADs.
#      The integers in that list are the instructions
#      that the interpreter executes when you call add().
#      There's actually a built-in module for looking at bytecode.
#      Why don't you import 'dis'?

# import this

# Scott: <starts reading>
# Joe: No, not 'this': dis.  It's the 'disassembly' module.
# Scott: Ah.

import dis

# Joe: You can do `dis.dis` of a function to show the disassembly.

dis.dis(add)

# Scott: What does all this mean?

# Joe: The dis function prints a human-readable representation 
#      of the instructions in the code object.
# 
#      1. There are 8 bytes in that code object, but there are only 4 instructions:
#         LOAD_FAST, LOAD_FAST, BINARY_ADD, RETURN_VALUE.

#      2. The first three bytes in the list correspond to the first LOAD_FAST. 
#         The second three bytes
#         correspond to the second LOAD_FAST, the last two bytes correspond to the BINARY_ADD
#         and RETURN_VALUE, respectively.

# Scott: Why does LOAD_FAST get three bytes, but BINARY_ADD and RETURN_VALUE only get one?

# Joe: 3. The two bytes after 124 are the **arguments** to the LOAD_FAST instruction. They
#         tell the interpreter which local variable to load.  
#         (124, 0, 0) says to load local variable zero.
#         (124, 1, 0) says to load local variable one.
#
#      4. In general, whenever an instruction takes an argument, 
#         the next two bytes contain the argument to the instruction.

# Scott: Why are those one and zero? Don't we want to load a and b.

# Joe:   Argument instructions are usually represented as little-endian 16-bit integers.
#        For instructions with an argument, the last two dis columns show the value of the
#        argument as an integer followed by the actual meaning of the argument.
#
#        In this case, dis says the first local variable is 'a' and the second local 
#        variable is 'b'.

# Scott: Okay. So LOAD_FAST of 0 tells Python to load 'a'. 
#        Where exactly are we loading it **to**?

# Joe:   LOAD instructions push values onto a stack where they can be 
#        manipulated by other instructions. The BINARY_ADD instruction 
#        doesn't have an argument in the bytecode because it always pops 
#        the top two values off the stack, adds them, and pushes the result 
#        back onto the stack.

# Scott: I think I understand:
# 
#        At the start of the function the stack is empty, we load a and b, 
#        and then we get to the BINARY_ADD. When we execute the add, we
#        pop 'a' and 'b' off the stack, add them together, 
#        and push the result back onto the stack. 
#        Finally, we hit the RETURN_VALUE instruction, which pops the top 
#        value off the stack and returns it to the caller.

# Joe: Perf.

# Scott: What are those numbers next to the instruction names?

# Joe: Those tell you the index in the bytecode of the start of that instruction. 
#      The first instruction starts at index 0, and the second instruction starts at index 3
#      because indices 1 and 2 were used for arguments.

# Scott: And 6 and 7 are next to each other because BINARY_ADD is one-byte instruction?

# Joe: Yep.

# Scott: What's the number on the top left

# Joe: That's the line number for the instructions on or below that line.
#      It's a little easier to see on a function with multiple lines.

[124, 0, 0, 124, 1, 0, 23, 83]
 17           0 LOAD_FAST                0 (a)
              3 LOAD_FAST                1 (b)
              6 BINARY_ADD
              7 RETURN_VALUE


In [8]:
# Scott: Ok...how about this?

def add_with_assign(a, b):
    x = a + b
    return x
dis.dis(add_with_assign)

# Scott: So this says that the first four instructions correspond to the fourth Python line,
#        and the next two instructions correspond to the 5th Python line.

# Joe: Yep. Why don't you try something a little more complex.

  4           0 LOAD_FAST                0 (a)
              3 LOAD_FAST                1 (b)
              6 BINARY_ADD
              7 STORE_FAST               2 (x)

  5          10 LOAD_FAST                2 (x)
             13 RETURN_VALUE


In [9]:
def abs(x):
    if x > 0:
        return x
    else:
        return -x
dis.dis(abs)

# Scott: <talk about LOAD_CONST> Argument is 1, but the value is zero.
#        <talk about COMPARE_OP> How does Python know that 4 means "greater than"?

# Joe: Not all instruction arguments are indices. 
#      COMPARE_OP takes an enum value as its argument.  
#      There are entries for less than, equal, not equal, etc.
#      That next instruction:
#
#      POP_JUMP_IF_FALSE does exactly what it says: it pops the top value off the stack and
#      jumps to the instruction at the index of its argument if that value is falsey.
#      Otherwise it continues onto the next instruction as normal.
#
# Scott: Okay. So when we get to the POP_JUMP_IF_FALSE, if the result of COMPARE_OP 
#        is truthy, we continue on to the next instruction. Otherwise, 
#        we jump to LOAD_FAST at index 16?

# Joe: Yep. Those arrows are dis' way of noting that byte 16 is a jump target.

# Scott: Okay. I think I understand the first branch, and I understand the second branch.
#        What about the final LOAD_CONST/RETURN_VALUE? 
#        I don't think we can ever hit those instructions.

# Joe: Yep, that's dead code.
#      CPython uses a fairly simple code generation algorithm.  
#      One of the rules is that if the body of a function doesn't end in a return statement, 
#      an implicit return of None is always inserted. In this case, even though it looks like
#      our function ends in a return value, CPython considers the if-statement to be
#      the last statement, so we get the dummy return even though it'll never be hit.

# Scott: That seems kinda wasteful...

# Joe: In most programs, an extra four unused bytes at the end of a function isn't a big deal.
#      That's only half the size of a pointer.
#      The CPython team decided that eliminating those four bytes isn't worth
#      the additional complexity it would add to the compiler.

# Scott: Okay, I guess that makes sense. 
#        Is there any way we could remove that code if we wanted to?

# Joe: Well...you don't **have** to use the CPython compiler to make a code object. You can
#      just construct one yourself like any other object.

  2           0 LOAD_FAST                0 (x)
              3 LOAD_CONST               1 (0)
              6 COMPARE_OP               4 (>)
              9 POP_JUMP_IF_FALSE       16

  3          12 LOAD_FAST                0 (x)
             15 RETURN_VALUE

  5     >>   16 LOAD_FAST                0 (x)
             19 UNARY_NEGATIVE
             20 RETURN_VALUE
             21 LOAD_CONST               0 (None)
             24 RETURN_VALUE


In [26]:
# Scott: Alright, well, let's write our own abs then!
# Joe:   Hold on there killer. Why don't I get you started with something a little 
#        simpler. How about addone?
# Scott: Alright, fine.  Let's write it out in Python first so we know what we're going for:

def addone(x):
    return x + 1

# Scott: Okay, so that's our target. You said I need to construct a code object.
#        Where do I find a code constructor?

# Joe:   The types module provides a CodeType.

# from types import CodeType
# print(CodeType.__doc__)

# Scott: Not for the faint of heart...great.

# my_code = CodeType(1, # argcount (Scott)
#                    0, # kwonlyargcount (Scott)
#                    1, # nlocals (Joe)
#                    2, # stacksize (Joe)
                   
# Joe: stacksize tells Python how much space to allocate for storing values on the stack.
#      We need enough slots to hold the maximum number of objects that ever 
#      will appear on the stack simultaneously.
#
# Scott: Okay, so, the biggest the stack is ever going to be is right before the add when
#        both 'x' and 1 are on the stack. So the stacksize should be two.
                   
# Scott: The next entry is `flags`.  What's the deal with those?

# Joe: `flags` is a bitmask of options for the code object.  
#      I've prepared some material to illustrate the flags in depth.
#      If you could be so kind as to press the down arrow on your keyboard.

# Scott: (presses down, then looks up, a beat)
# Scott: (confused sputtering)
# Joe: (brushing off Scott's confusion) Let's keep on track here...if 
#      take a look at the slide, you can see that there's a lot of different options.

In [15]:
# Used to determine if certain extra optimizations can be made.
# In practice, means "is this code for a function (as opposed to a module or class body)".
CO_OPTIMIZED          = 0x0001

# Should a new locals dict be allocated every time this code is executed?
CO_NEWLOCALS          = 0x0002

CO_VARARGS            = 0x0004
CO_VARKEYWORDS        = 0x0008

# Are we defined inside another function?
CO_NESTED             = 0x0010

# Are we a generator?
CO_GENERATOR          = 0x0020

# Do we share any variable cells with another function.
# We could infer this from other info on the code object,
# but this used for optimization purposes by the interpreter.
CO_NOFREE             = 0x0040

# Are we an async-def'd coroutine or a types.coroutine?
CO_COROUTINE          = 0x0080
CO_ITERABLE_COROUTINE = 0x0100

In [22]:
# __future__ flags
CO_FUTURE_DIVISION         = 0x2000
CO_FUTURE_ABSOLUTE_IMPORT  = 0x4000
CO_FUTURE_WITH_STATEMENT   = 0x8000
CO_FUTURE_PRINT_FUNCTION   = 0x10000
CO_FUTURE_UNICODE_LITERALS = 0x20000

# Compiled with enhanced inequality syntax.
CO_FUTURE_BARRY_AS_BDFL    = 0x40000

# Python 3.5 backwards-compat flag.
CO_FUTURE_GENERATOR_STOP   = 0x80000

In [17]:
# code(argcount, kwonlyargcount, nlocals, stacksize, flags, codestring,
#      constants, names, varnames, filename, name, firstlineno,
#      lnotab[, freevars[, cellvars]])

my_code = CodeType(1,             # argcount
                   0,             # kwonlyargcount
                   1,             # nlocals
                   2,             # stacksize
                   (CO_OPTIMIZED | CO_NEWLOCALS | CO_NOFREE),
# (Resume Typing)
                   bytes([124, 0, 0, 100, 0, 0, 23, 83]),
                   (1,),          # constants
                   (),            # names
                   ('x',),        # varnames
                   '<string>',    # filename
                   'addone',      # name
                   42,            # firstlineno
                   b"",           # lnotab
                   (),            # freevars
                   ())            # cellvars

# Scott: Okay, codestring. (types, 'bytes([])') What are our bytes?

# Joe: (quickly, but not memorized) 124, 0, 0, 100, 0, 0, 23, 83

# Scott: Could you elaborate a little bit on that?

# Joe: Well, looking at our function, we need to load 'x', load 1, do a binary add, 
#      and then return the result. To load x, we need a LOAD_FAST instruction.
#      The opcode for LOAD_FAST is 124. 'x' is the only local variable, so the argument is 0.
#      Next we want to load 1 with a LOAD_CONST instruction. The opcode for LOAD_CONST is 100.
#      We're only using one constant, so we can put the value 1 at index 0.
#      We saw earlier that the opcodes for BINARY_ADD and RETURN_VALUE were 23 and 83.

# Scott: It's impressive that you can just rattle that off.

# Joe: Well, you should expect nothing less from a certified bytecode expert.

# Scott: Okay. So if we're doing a LOAD_CONST of zero, I guess we want a tuple containing just one?

# Joe: (thumbs up)

# Scott: Okay, names?

# Joe: Names should be a tuple containing the names of global variables and 
#      attributes that appear in the function.
#      We don't have any, so you just want an empty tuple.

# Scott: One empty tuple, coming right up. Okay...varnames. How is that different from names?

# Joe: Varnames is a tuple containing the names of the local variables for the function, 
#      in the order that they're indexed by LOAD_FAST and STORE_FAST.

# Scott: So that's just 'x'? (types `('x',))

# Joe: (affirmative noises)

# Joe: The next few entries don't really matter for a hand-written code object:
#      filename is normally the name of the file where the code was defined.
#      We don't have a file, but the convention for exec is the word 'string' 
#      in angle brackets. (Scott types)
#      Next is the name of the code object. (Scott types 'addone')
#      Then we have the first line number where the instructions appear. 
#      Pick your favorite number.
#      After that is the line number table. It's a bytes object representing a mapping 
#      from instructions to their line number. We don't really care about line numbers here,
#      so you can just put an empty bytes object.

# Joe: Last but not least, we have the freevars and the cellvars. These are the names of 
#      variables that are shared with other functions. 
#      Since we set CO_NOFREE, those should be empty tuples. (Scott types)

In [23]:
dis.dis(my_code)

 42           0 LOAD_FAST                0 (x)
              3 LOAD_CONST               0 (1)
              6 BINARY_ADD
              7 RETURN_VALUE


In [24]:
my_code(1)

TypeError: 'code' object is not callable

In [20]:
from types import FunctionType
FunctionType?

my_addone = FunctionType(my_code, {})

In [21]:
my_addone(1)

2